In [ ]:
# @title 🚀 1. LIMPAR E ATIVAR GERADOR
import os
!rm -f engine.py engine.py.*
!pip install -q diffusers transformers accelerate flask flask-cors
!wget -q https://raw.githubusercontent.com/Miraplay2025/Gerador-de-imagem/main/engine.py

import engine
print("\n🟢 Motor Limpo e Reiniciado! Clique no Play da célula abaixo.")

In [ ]:
# @title 🔗 2. GERAR URL PÚBLICA (CLOUDFLARE)
import os, time, subprocess, re

print("📥 Preparando Cloudflare...")
!rm -f cloudflared-linux-amd64.deb*
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

print("🌐 Abrindo túnel... Aguarde o link aparecer.")
os.system("pkill cloudflared")

# Inicia o cloudflared capturando o output para encontrar o link
p = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:5000'], 
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Loop para ler o log e encontrar a URL .trycloudflare.com
for line in p.stdout:
    if ".trycloudflare.com" in line:
        url = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
        if url:
            print("\n" + "="*50)
            print(f"✅ LINK DO SEU GERADOR: {url.group(0)}")
            print("="*50 + "\n")
            break
    if "Error" in line:
        print(line)